# Doubly Linked List

In [31]:
# create a new node
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None
        self.prev = None

# constructor of a linked list
class LinkedList:
    def __init__(self, value):
        # create a new node using the Node class
        new_node = Node(value)

        # point the head to the new node
        self.head = new_node

        # point the tail to the new node
        self.tail = new_node

        # start the linked list of length = 1
        self.length = 1

    def print_list(self):
        temp = self.head
        while temp:
            print(temp.value)
            temp = temp.next

    def append(self, value):
        # create a new node with value
        new_node = Node(value)

        # if list is empty, assign head and tail to new node
        if self.head is None:
            self.head = new_node
            self.tail = new_node
        
        else:
            # if list is not empty, first assign new_node to tail.next (append)
            self.tail.next = new_node
            new_node.prev = self.tail

            # then point the tail to the new node 
            self.tail = new_node

        # increase the length by 1
        self.length += 1

    def pop(self):
        # point temp to the tail
        temp = self.tail

        # if list if empty, return None
        if self.length == 0:
            return None

        # for 1 item
        if self.length == 1:
            self.head = None
            self.tail = None

        # for 2 or more items, assign the prev node of tail as the new tail node and the next value as None, then detach the temp node from the list by assigning its prev to None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
            temp.prev = None

        # decrement the length by 1
        self.length -= 1
        return temp

    def prepend(self, value):
        new_node = Node(value)

        if self.length == 0:
            self.head = new_node
            self.tail = new_node

        else:
            new_node.next = self.head
            self.head.prev = new_node
            self.head = new_node

        self.length += 1


    def pop_first(self):
        if self.length == 0:
            return None

        elif self.length == 1:
            self.head = None
            self.tail = None

        else:
            temp = self.head
            self.head = self.head.next
            temp.next = None
        
        self.length -= 1

    def get(self, index):
        # check if index is within bounds
        if index<0 or index>=self.length:
            return None

        # point temp to the head
        temp = self.head

        # move temp until index is out of bounds
        for _ in range(index):
            temp = temp.next

        return temp

    def set_value(self, value, index):

        temp = self.get(index)

        if temp:
            temp.value = value
            return True
        
        else:
            return False

    def insert(self, value, index):
        if index<0 or index>self.length:
            return False

        if index == 0:
            return self.prepend(value)

        if index == self.length:
            return self.append(value)

        new_node = Node(value)
        temp = self.get(index-1)
        new_node.next = temp.next
        temp.next = new_node

        self.length += 1
        return True

    def remove(self, index):
        if index<0 or index>=self.length:
            return None

        if index == 0:
            return self.pop_first()

        if index == self.length - 1:
            return self.pop()

        prev = self.get(index - 1)
        temp = prev.next

        prev.next = temp.next
        temp.next = None

        self.length -= 1
        return temp

    
    def reverse(self):
        temp = self.head
        self.head = self.tail
        self.tail = temp

        after_temp = temp.next
        before_temp = None
        
        for _ in range(self.length):
            # inital position of before, temp and after
            after_temp = temp.next

            # start reversing: temp points to before
            temp.next = before_temp

            # start reversing: move before pointer to temp 
            before_temp = temp

            # start reversing: move temp pointer to after 
            temp = after_temp



In [32]:
# create a linked list, point the head and tail and assign the value of 4 to the new node
my_linked_list = LinkedList(1)
my_linked_list.append(2)
my_linked_list.append(3)

my_linked_list.reverse()
my_linked_list.print_list()



3
2
1


Build a cache data structure which implements the following eviction policy:

1. evict an expired item if it exists
2. otherwise find the items with the lowest priority and evict the Least Recently Used of these items

In [ ]:
from collections import defaultdict

class Node:
    def __init__(self, key, value, priority):
        self.key = key
        self.value = value
        self.priority = priority
        self.prev = None
        self.next = None

class Cache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = {}
        self.priority_map = defaultdict(list)
        self.head = None
        self.tail = None

    def get(self, key):
        if key in self.cache:
            node = self.cache[key]
            self._move_to_front(node)
            return node.value
        else:
            return None

    def put(self, key, value, priority, expiration=None):
        if key in self.cache:
            node = self.cache[key]
            node.value = value
            node.priority = priority
            self._move_to_front(node)
        else:
            if len(self.cache) >= self.capacity:
                self._evict()
            node = Node(key, value, priority)
            self.cache[key] = node
            self._add_to_front(node)
        if expiration is not None:
            self.priority_map[expiration].append(node)

    def _move_to_front(self, node):
        if node == self.head:
            return
        if node == self.tail:
            self.tail = node.prev
            self.tail.next = None
        else:
            node.prev.next = node.next
            node.next.prev = node.prev
        self._add_to_front(node)

    def _add_to_front(self, node):
        if self.head is None:
            self.head = node
            self.tail = node
        else:
            node.next = self.head
            self.head.prev = node
            self.head = node

    def _evict(self):
        if len(self.priority_map) > 0:
            min_priority = min(self.priority_map.keys())
            nodes_with_lowest_priority = self.priority_map[min_priority]
            node_to_evict = nodes_with_lowest_priority.pop(0)
            del self.cache[node_to_evict.key]
            if len(nodes_with_lowest_priority) == 0:
                del self.priority_map[min_priority]
        else:
            node_to_evict = self.tail
            del self.cache[node_to_evict.key]
            if self.head == self.tail:
                self.head = None
                self.tail = None
            else:
                self.tail = self.tail.prev
                self.tail.next = None

    def __repr__(self):
        items = []
        current = self.head
        while current:
            items.append((current.key, current.value))
            current = current.next
        return f"Cache({items})"
